# 08 - Ejercicio 1 EDA - Datos del Naufragio del Titanic


* Para este ejercicio disponemos de una serie de datos relativos a los ***pasajeros que se embarcaron en el Titanic***, así como un flag que nos indica si ***sobrevivieron o no***.


* Estos datos se encuentran en los siguientes ficheros:

    - ***Pasajeros***: ./data/titanic/pasajeros.csv
    - ***Supervivientes***: ./data/titanic/supervivientes.csv


* Los ***campos que hay en el fichero de Pasajeros*** son los siguientes:
    - PassengerId: identificador de pasajero
    - Pclass: clase en la que viajaba 1 = 1st, 2 = 2nd, 3 = 3rd
    - Name
    - Sex
    - Age
    - SibSp: hermanos o conyujes abordo
    - Parch: padres o hijos abordo
    - Ticket: id del ticket
    - Fare
    - Cabin: id de la cabina
    - Embarked: Puerto de embarcación C = Cherbourg, Q = Queenstown, S = Southampton

* Los ***campos que hay en el fichero de Supervivientes*** son los siguientes:
    - PassengerId:  identificador de pasajero
    - Survived: 1: sobrevivio, 0: murió

#### Para este ejercicio se pide:

* Dado un DataFrame (df) ya cargado con los datos de los pasajeros y si sobrevivieron o no:

    1. ***Crear una nueva columna en el DataFrame (df) llamada puerto, que contenga en Nombre del puerto*** (No el identificador del puerto)
    2. ***Obtener el número de pasajeros que embarcaron en cada puerto*** (Cherbourg, Queenstown, Southampton)
    3. ***Obtener cuantos hombres y mujeres embarcaron***
    4. ***Obtener la edad media de los hombres y mujeres que sobrevivieron y murieron***
    5. ***Obtener cuantos muertos hubo por rango de edad*** (< 18 años: jovenes, 18-65 años: adultos, 65 >: Anciano) 
    6. ***Obtener el número de muertos por clase y genero***
    7. ***Obtener el número de muertos y supervivientes por ciudad de origen***

<hr>

#### Pasos previos:

   - Importamos la librería de pandas
   - Cargamos los ficheros y los pasamos a un DataFrame
   - Creamos el DataFrame para realizar los ejercicios, realizando un ***inner join*** por el campo *'PassengerId'*

In [1]:
import pandas as pd

# Cargamos los ficheros
df_pasajeros = pd.read_csv('../data/titanic/pasajeros.csv', header=0)
df_supervivientes = pd.read_csv('../data/titanic/supervivientes.csv', header=0)

# Join
df = pd.merge(df_pasajeros, df_supervivientes, on='PassengerId', how='inner')

df.sample(5)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
201,202,3,"Sage, Mr. Frederick",male,NaN,8,2,CA. 2343,69.55,NaN,S,0.0
758,759,3,"Theobald, Mr. Thomas Leonard",male,34.0,0,0,363294,8.05,NaN,S,0.0
945,946,2,"Mangiavacchi, Mr. Serafino Emilio",male,NaN,0,0,SC/A.3 2861,15.5792,NaN,C,0.0
1015,1016,3,"Kennedy, Mr. John",male,NaN,0,0,368783,7.75,NaN,Q,0.0
356,357,1,"Bowerman, Miss. Elsie Edith",female,22.0,0,1,113505,55,E33,S,1.0


<hr>


#### 1. ***Crear una nueva columna en el DataFrame (df) llamada puerto, que contenga en Nombre del puerto*** (No el identificador del puerto)

In [2]:
df['puerto'] = (df['Embarked']
                .apply(lambda x: 'Cherbourg' if x== 'C' 
                       else ('Southampton' if x=='S' 
                             else 'Queenstown')))
df.sample(5)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived,puerto
296,297,3,"Hanna, Mr. Mansour",male,23.5,0,0,2693,7.2292,NaN,C,0.0,Cherbourg
592,593,3,"Elsbury, Mr. William James",male,47.0,0,0,A/5 3902,7.25,NaN,S,0.0,Southampton
1126,1127,3,"Vendel, Mr. Olof Edvin",male,20.0,0,0,350416,7.8542,NaN,S,0.0,Southampton
897,898,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,NaN,Q,1.0,Queenstown
209,210,1,"Blank, Mr. Henry",male,40.0,0,0,112277,31,A31,C,1.0,Cherbourg


<hr>


#### 2. ***Obtener el número de pasajeros que embarcaron en cada puerto*** (Cherbourg, Queenstown, Southampton)

In [3]:
(df
 .groupby(['puerto'])['puerto']
 .agg(['count'])
 .reset_index())

,puerto,count
0,Cherbourg,269
1,Queenstown,126
2,Southampton,914


<hr>


#### 3. ***Obtener cuantos hombres y mujeres embarcaron***

In [4]:
(df
 .groupby(['Sex'])['Sex']
 .agg(['count'])
 .reset_index())

,Sex,count
0,female,466
1,male,843


<hr>


#### 4. Obtener la edad media de los hombres y mujeres que sobrevivieron y murieron

In [5]:
(df
 .groupby(['Sex', 'Survived'])['Age']
 .agg(['mean'])
 .reset_index())

,Sex,Survived,mean
0,female,0.0,25.046875
1,female,1.0,29.406142
2,male,0.0,31.129929
3,male,1.0,27.276022


<hr>


#### 5. ***Obtener cuantos muertos hubo por rango de edad*** (< 18 años: jovenes, 18-65 años: adultos, 65 >: Anciano)


* En primer lugar creamos una ***nueva columna*** llamada ***'rango_edad'***.


* Filtramos los muertos y los de edad nula.


* Agregamos por la columna ***'rango_edad'*** y realizamos un ***conteo***.

In [6]:
# Nueva Columna
df['rango_edad'] = (df['Age']
                    .apply(lambda x: 'Joven' if x < 18 
                           else ('Anciano' if x > 65 
                                 else ('Adulto' if x >= 18 and x <=65 else 'NULO'))))



# Filtramos y Agregacion
(df[(df['Survived'] == 0) & (df['rango_edad'] != 'NULO')]
 .groupby(['rango_edad'])['rango_edad']
 .agg(['count']))

,count
rango_edad,
Adulto,545
Anciano,8
Joven,76


<hr>


#### 6. ***Obtener el número de muertos por clase y genero***


* En primer lugar tenemos que ***filtrar*** todos aquellos pasajeros que murieron.


* En segundo lugar ***agregamos y realizamos el conteo***.

In [7]:
(df[df['Survived'] == 1]
 .groupby(['Pclass', 'Sex'])['Survived']
 .agg(['count'])
 .reset_index())

,Pclass,Sex,count
0,1,female,141
1,1,male,45
2,2,female,100
3,2,male,17
4,3,female,144
5,3,male,47


<hr>


#### 7. ***Obtener el número de muertos y supervivientes por ciudad de origen***

In [8]:
(df
 .groupby(['Survived', 'puerto'])['Survived']
 .agg(['count'])
 .reset_index())

,Survived,puerto,count
0,0.0,Cherbourg,137
1,0.0,Queenstown,69
2,0.0,Southampton,609
3,1.0,Cherbourg,132
4,1.0,Queenstown,57
5,1.0,Southampton,305


<hr>

*Este Notebook ha sido desarrollado por **Ricardo Moya García** y registrado en Safe Creative como ***Atribución-NoComercial-CompartirIgual***.*

<img src="../imgs/CC_BY-NC-SA.png" alt="CC BY-NC">